In [1]:
#this is final one from spring 2022 

import numpy as np #basic python library
import matplotlib.pyplot as plt #For plotting results
from matplotlib.pyplot import figure, legend, plot, show, title, xlabel, ylabel, ylim
import cv2 
import glob

## Import Landlab components
from landlab.components import DepressionFinderAndRouter #Pit filling
from landlab.components import PriorityFloodFlowRouter #Flow routing
from landlab.components import FlowAccumulator #FlowAccumulator 
from landlab.components import ChannelProfiler 
from landlab.components import SteepnessFinder
from landlab.components import ChiFinder
from landlab.components import GroundwaterDupuitPercolator

#SPACE model
from landlab.components import Space #SPACE model
from landlab.components import SpaceLargeScaleEroder #basically SPACE 2.0 -- use this 
from landlab.components import FastscapeEroder #calculates the amount of erosion at each node
from landlab.components import SinkFillerBarnes #replaces pits with shallow gradients for draining

## Import Landlab utilities
from landlab import RasterModelGrid #Grid utility
from landlab import imshow_grid #For plotting results
from landlab.io import read_esri_ascii #to read in an ascii file
from landlab.io import read_asc_header #to read in the header of the ascii file

%matplotlib inline


In [2]:
#Set grid parameters
num_rows = 50
num_columns = 50
node_spacing = 10.00
node_next_to_outlet = num_columns + 1
outlet_node = 0 

#Set Model Time Parameters
timestep = 100.0
run_time = 5000 #years

uplift_rate = .001 # m/year
uplift_per_timestep = uplift_rate * timestep

#Instantiate model grid
mg = RasterModelGrid((num_rows, num_columns), node_spacing)

#Close all model boundary edges
mg.set_closed_boundaries_at_grid_edges(bottom_is_closed=True,left_is_closed=True,right_is_closed=True,top_is_closed=True)
mg.status_at_node[outlet_node] = mg.BC_NODE_IS_FIXED_VALUE

#Add field 'soil__depth' to the grid
mg.add_zeros('node', 'soil__depth')

#Set 2 m of initial soil depth at core nodes
mg.at_node['soil__depth'][:] = 2 #meters 

#topographic roughness
#random_noise = np.random.rand(len(mg.node_y)) /1000 #impose topography values on model grid

#Set constant random seed for consistent topographic roughness
np.random.seed(seed = 5000)

#Add field 'bedrock__elevation' to the grid
mg.add_zeros("bedrock__elevation", at="node")

#Sum 'soil__depth' and 'bedrock__elevation' to yield 'topographic elevation' 
#when u change spacing high

mg.at_node["bedrock__elevation"][:] += (mg.node_y * .00001 + mg.node_x * .00001 + np.random.rand(len(mg.node_y)) / 100) #cm scale rn 
mg.at_node["bedrock__elevation"][outlet_node] = 0 # do this before the mg.at node stuff above

#add aquifer
mg.add_zeros("node", "aquifer_base__elevation")
mg.at_node("aquifer_base__elevation")[:] += mg.at_node["bedrock__elevation"]
mg.add_zeros('node', 'water_table__elevation') 
mg.at_node('water_table__elevation')[:] += mg.at_node["aquifer_base__elevation"] + 3

#Add field ’topographic elevation’ to the grid
mg.add_zeros('node', 'topographic__elevation')
mg.at_node["topographic__elevation"][:] += mg.at_node["bedrock__elevation"]# + mg.at_node['soil__depth'][:]
mg.at_node["topographic__elevation"][:] += mg.at_node["soil__depth"] + mg.at_node["water_table__elevation"]
#mg.at_node["topographic__elevation"][:] += mg.at_node["soil__depth"]

#outlet_node = mg.set_watershed_boundary_condition_outlet_id(0,mg['node']['topographic__elevation'], -9999.)
#mg.at_node['bedrock__elevation'][outlet_node] = 1 # do this before the mg.at node stuff above
#outlet_node = mg.set_watershed_boundary_condition_outlet_id(0,mg['node']['topographic__elevation'], -9999.)

mg.at_node['topographic__elevation'][outlet_node] = 0
mg.at_node['soil__depth'][outlet_node] = 0
mg.at_node['water_table__elevation'] = 0
print(outlet_node)

TypeError: 'FieldDataset' object is not callable

In [ ]:
#Parameters for SPACE and Fastscape
K_sed = 0.0001
K_sp = 0.001
K_br = 0.0001 # depends on size of spatial scale #doubles it from .00001 to get it quicker
F_f = 0.0
phi = 0.0
H_star = 1.0
v_s = 5.0
m_sp = 0.5
n_sp = 1.0
sp_crit_sed = 0
sp_crit_br = 0

#Instantiate the Flow accumulator
fr = PriorityFloodFlowRouter(mg, flow_metric='D8', suppress_out = True)

#Instantiate the Flow accumulator
fa = FlowAccumulator(mg, flow_director='D8') 

#Instantiate the depressionfinder and router as df
df = DepressionFinderAndRouter(mg,pits = 'flow__sink_flag', reroute_flow = True) 

#instantiate the fascape eroder as fsc 
fsc = FastscapeEroder(mg, K_sp, m_sp, n_sp)  

#Instantiate SPACE model with chosen parameters as sp
sp = SpaceLargeScaleEroder(mg, K_sed, K_br, F_f, phi, H_star, v_s, m_sp, n_sp, sp_crit_sed, sp_crit_br)

#instantiate groundwater
gdp = GroundwaterDupuitPercolator(mg, recharge_rate=1E-7)

Plot of Topography (pre fastscape)

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)


Plot of Water Table (pre fastscape)

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'water_table__elevation', plot_name='Water Table Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)


Run Fastscape

In [ ]:
#run for 10 years
for x in range(run_time):
    #core nodes are anything except outer boundary
    mg.at_node['bedrock__elevation'][mg.core_nodes] += uplift_per_timestep  #adding uplift to the bedrock
    mg.at_node['topographic__elevation'] = mg.at_node['bedrock__elevation'] + mg.at_node['soil__depth']  + mg.at_node['water_table__elevation']#changing the elevation to account for the uplift
    #DONT ADD MG.CORE_NODES OR : AFTER TOPO ELEVATION
    #mg.at_node["topographic__elevation"][0]-= 0.001 # Uplift
    #mg.at_node["bedrock__elevation"][0]-= 0.001 # Uplift

    if x ==1000:
        figelev = plt.figure()
        plot = plt.subplot()
        imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)
    gdp.run_one_step()
    fa.run_one_step()
    df.map_depressions()
    fsc.run_one_step(timestep)
    print(x)

    #looks like runtime is 1 minute every 10,000 years (or 100 x) with kbr = .00002

Plot of Topography (post fastscape)

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'topographic__elevation', plot_name='Topographic Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)

Plot of Water Table (post fastscape)

In [ ]:
#Instantiate figure as empty plot
figelev = plt.figure()

#Instantiate subplot as empty plot
plot = plt.subplot()

#Create a topographic elevation plot that shows the elevation of the landscape in diff colors - using landlab utility imshow_grid
imshow_grid(mg, 'water_table__elevation', plot_name='Water Table Elevation', var_name = 'Elevation', var_units=r'm', grid_units=('m', 'm'), cmap='terrain',color_for_background=None)